In [28]:
# Exploratory Data Analysis (EDA)
import pandas as pd
import plotly.express as px

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Hyperparameter Tuning
import optuna

# Set Pandas and Plotly display options
pd.set_option("display.float_format", lambda x: "%.2f" % x)
# px.defaults.template = "plotly_dark"

In [29]:
df = pd.read_csv("./data/clients.csv")
df.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


## Exploratory Data Analysis


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [31]:
# Distribuição de inovação
inovacao_perc = df["inovacao"].value_counts(normalize=True) * 100
px.bar(
    x=inovacao_perc.index,
    y=inovacao_perc.values,
    color=inovacao_perc.index,
    labels={"x": "Inovação", "y": "Percentual (%)"},
    title="Distribuição de Inovação",
)

Aplica um **teste de ANOVA** (Análise de Variância) para verificar se há variações significativas na média de **faturamento mensal** para diferentes níveis de inovação.

**Pressupostos:**

- As amostras são independentes.
- As populações têm distribuições normais.
- As variâncias das populações são iguais (homocedasticidade).
- As amostras são de tamanhos iguais.


In [32]:
# Verifica se os dados são homogêneos usando um teste de Bartlett. O teste de Bartlett é um teste de
# hipótese que verifica se as variâncias dos grupos são iguais. Se o valor-p for menor que 0,05, as
# variâncias são consideradas diferentes (os dados não são homogêneos).
# - H0: As variâncias são iguais.
# - H1: As variâncias são diferentes.
from scipy.stats import bartlett

# Agrupar os dados de faturamento mensal por nível de inovação
grouped_incomes = df.groupby("inovacao")["faturamento_mensal"].apply(list)

# Teste de Bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*grouped_incomes)
print(f"Teste de Bartlett: {bartlett_p_value:.4f}")
print(
    "As variâncias são iguais."
    if bartlett_p_value > 0.05
    else "As variâncias são diferentes."
)

Teste de Bartlett: 0.2825
As variâncias são iguais.


In [33]:
# Aplica um teste de normalidade de Shapiro-Wilk para verificar se os dados seguem uma distribuição
# normal. Se o valor-p for menor que 0,05, os dados não seguem uma distribuição normal.
# - H0: Os dados seguem uma distribuição normal.
# - H1: Os dados não seguem uma distribuição normal.
from scipy.stats import shapiro

# Teste de Shapiro-Wilk
shapiro_test_statistic, shapiro_p_value = shapiro(df["faturamento_mensal"])
print(f"Teste de Shapiro-Wilk: {shapiro_p_value:.4f}")
print(
    "Os dados seguem uma distribuição normal."
    if shapiro_p_value > 0.05
    else "Os dados não seguem uma distribuição normal."
)

Teste de Shapiro-Wilk: 0.2351
Os dados seguem uma distribuição normal.


In [34]:
# Aplica o teste de Welsh, pois as amostras são de tamanhos diferentes. O teste de Welch é um teste
# de hipótese que verifica se as médias dos grupos são iguais. Se o valor-p for menor que 0,05, as
# médias são consideradas diferentes.
# - H0: As médias são iguais.
# - H1: As médias são diferentes.
from pingouin import welch_anova

aov = welch_anova(data=df, dv="faturamento_mensal", between="inovacao")
aov_p_value = aov["p-unc"][0]

print(f"Teste de Welch: {aov_p_value:.4f}")
print("As médias são iguais." if aov_p_value > 0.05 else "As médias são diferentes.")

Teste de Welch: 0.3453
As médias são iguais.


## Training the model


In [35]:
# Use all columns as X because we don't have a target variable
X = df.copy()

# Separate variables into numerical, nominal, and ordinal
numeric_features = ["faturamento_mensal", "numero_de_funcionarios", "idade"]
nominal_features = ["localizacao", "atividade_economica"]
ordinal_features = ["inovacao"]

# Apply transformations to the data
numeric_transformer = StandardScaler()
nominal_transformer = OneHotEncoder(drop="first")
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("nom", nominal_transformer, nominal_features),
        ("ord", ordinal_transformer, ordinal_features),
    ]
)

# Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

In [36]:
# Optimize hyperparameters for KMeans using Optuna
distance_metrics = ["euclidean", "minkowski"]


def objective(trial: optuna.Trial):
    n_clusters = trial.suggest_int("n_clusters", 3, 10)
    distance_metric = trial.suggest_categorical("distance_metric", distance_metrics)

    kmeans = KMeans(n_clusters=n_clusters, random_state=51)
    kmeans.fit(X_transformed)

    distances = pairwise_distances(X_transformed, metric=distance_metric)
    silhouette_avg = silhouette_score(distances, kmeans.labels_, metric=distance_metric)

    return silhouette_avg


search_space = {"n_clusters": range(3, 11), "distance_metric": distance_metrics}
sampler = optuna.samplers.GridSampler(search_space)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective)

# Get the best hyperparameters
best_params = study.best_params
best_score = study.best_value
print(f"Best hyperparameters: {best_params}")
print(f"Best silhouette score: {best_score:.4f}")

[I 2024-09-23 07:16:16,395] A new study created in memory with name: no-name-66c1e750-8eea-4225-aab0-9e9ef642a641
[I 2024-09-23 07:16:16,405] Trial 0 finished with value: 0.39627770889555175 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.39627770889555175.
[I 2024-09-23 07:16:16,413] Trial 1 finished with value: 0.13619033234907765 and parameters: {'n_clusters': 9, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.39627770889555175.
[I 2024-09-23 07:16:16,436] Trial 2 finished with value: 0.4460089237295571 and parameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4460089237295571.
[I 2024-09-23 07:16:16,458] Trial 3 finished with value: 0.3962777088955517 and parameters: {'n_clusters': 4, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.4460089237295571.
[I 2024-09-23 07:16:16,481] Trial 4 finished with value: 0.15660746520773064 and parameters: {'n_clusters': 8, 'distan

Best hyperparameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}
Best silhouette score: 0.4460


In [37]:
n_clusters = best_params["n_clusters"]
distance_metric = best_params["distance_metric"]

# Train the model with the best hyperparameters
model = KMeans(n_clusters, random_state=51)
model.fit(X_transformed)

# Calculate the silhouette score
distances = pairwise_distances(X_transformed, metric=distance_metric)
silhouette = silhouette_score(distances, model.labels_, metric=distance_metric)

print(f"Number of clusters: {n_clusters}")
print(f"Distance metric: {distance_metric}")
print(f"Silhouette score: {silhouette:.4f}")

Number of clusters: 3
Distance metric: minkowski
Silhouette score: 0.4460


In [38]:
# Add cluster to the original dataframe
df["cluster"] = model.labels_

In [39]:
df.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


## Plotting the results


In [40]:
# Plot "idade" and "faturamento_mensal" by cluster
px.scatter(
    df,
    x="idade",
    y="faturamento_mensal",
    color="cluster",
    title="Idade vs. Faturamento Mensal por Cluster",
    labels={"idade": "Idade", "faturamento_mensal": "Faturamento Mensal"},
)

In [41]:
# Plot "inovacao" and "faturamento_mensal" by cluster
px.scatter(
    df,
    x="inovacao",
    y="faturamento_mensal",
    color="cluster",
    title="Inovação vs. Faturamento Mensal por Cluster",
    labels={"inovacao": "Inovação", "faturamento_mensal": "Faturamento Mensal"},
)

In [42]:
# Plot "inovacao" and "faturamento_mensal" by cluster
px.scatter(
    df,
    x="numero_de_funcionarios",
    y="faturamento_mensal",
    color="cluster",
    title="Numéro de funcionários vs. Faturamento Mensal por Cluster",
    labels={
        "numero_de_funcionarios": "Numéro de funcionários",
        "faturamento_mensal": "Faturamento Mensal",
    },
)

## Saving the model


In [43]:
import joblib
import os

os.makedirs("models", exist_ok=True)
joblib.dump(model, "models/clients_clustering_model.pkl")
joblib.dump(preprocessor, "models/clients_clustering_pipeline.pkl")

['models/clients_clustering_pipeline.pkl']

## Batch application with Gradio


In [44]:
import gradio as gr

model = joblib.load("models/clients_clustering_model.pkl")
preprocessor = joblib.load("models/clients_clustering_pipeline.pkl")


def predict_cluster(file):
    df = pd.read_csv(file.name)

    # Apply transformations to the data
    X_transformed = preprocessor.transform(df)

    # Predict the clusters
    df["cluster"] = model.predict(X_transformed)
    df.to_csv("./clusters.csv", index=False)

    return "./clusters.csv"

In [45]:
app = gr.Interface(
    fn=predict_cluster,
    inputs="file",
    outputs="file",
    title="Client Clustering",
    description="This app predicts the cluster of clients based on their features.",
)

app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
